# Training additional machine learning models for the task of enzyme-substrate pair prediction

### 1. Loading and preprocessing data for model training and evaluation
### 2. Training and validation machine learning models

In [8]:
import pandas as pd
import numpy as np
import random
import pickle
import sys
import os
import logging
from os.path import join
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
#from hyperopt import fmin, tpe, hp, Trials, rand
import xgboost as xgb
from sklearn.metrics import matthews_corrcoef
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
from hyperopt import fmin, tpe, hp, Trials, rand


sys.path.append('.\\additional_code')
#from data_preprocessing import *

CURRENT_DIR = os.getcwd()
print(CURRENT_DIR)

/root/XWY/esp/ESP/notebooks_and_code


## 1. Loading and preprocessing data for model training and evaluation

In [9]:
def array_column_to_strings(df, column):
    df[column] = [str(list(df[column][ind])) for ind in df.index]
    return(df)

def string_column_to_array(df, column):
    df[column] = [np.array(eval(df[column][ind])) for ind in df.index]
    return(df)

### (a) Loading data: 
Only keeping data points from the GO Annotation database with experimental evidence

In [10]:
df_train = pd.read_pickle(join(CURRENT_DIR, ".." ,"data","splits", "df_train_with_ESM1b_ts_GNN.pkl"))
df_train = df_train.loc[df_train["ESM1b"] != ""]
df_train = df_train.loc[df_train["type"] != "engqvist"]
df_train = df_train.loc[df_train["GNN rep"] != ""]
df_train.reset_index(inplace = True, drop = True)

df_test  = pd.read_pickle(join(CURRENT_DIR, ".." ,"data","splits", "df_test_with_ESM1b_ts_GNN.pkl"))
df_test = df_test.loc[df_test["ESM1b"] != ""]
df_test = df_test.loc[df_test["type"] != "engqvist"]
df_test = df_test.loc[df_test["GNN rep"] != ""]
df_test.reset_index(inplace = True, drop = True)
print(len(df_train), len(df_test))

df_unimol = pd.read_pickle(join(CURRENT_DIR, ".." ,"data","substrate_data", "df_unimol.pkl"))

df_merged = pd.merge(df_train, df_unimol[['substrate ID', 'unimol']], on='substrate ID', how='left')
df_train = df_merged.loc[pd.notna(df_merged["unimol"]) & (df_merged["unimol"] != "")]

df_merged = pd.merge(df_test, df_unimol[['substrate ID', 'unimol']], on='substrate ID', how='left')
df_test = df_merged.loc[pd.notna(df_merged["unimol"]) & (df_merged["unimol"] != "")]
print(len(df_train), len(df_test))
df_train.reset_index(inplace = True, drop = True)
df_test.reset_index(inplace = True, drop = True)

df_train.keys()
train_length, test_length = len(df_train), len(df_test)

/opt/anaconda3/envs/esp_38/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)
/opt/anaconda3/envs/esp_38/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


55878 13459
52619 12614


### (b) Splitting training set into 5-folds for hyperparameter optimization:
The 5 folds are created in such a way that the same enzyme does not occure in two different folds

In [4]:
def split_dataframe(df, frac):
    df1 = pd.DataFrame(columns = list(df.columns))
    df2 = pd.DataFrame(columns = list(df.columns))
    try:
        df.drop(columns = ["level_0"], inplace = True)
    except: 
        pass
    df.reset_index(inplace = True)
    
    train_indices = []
    test_indices = []
    ind = 0
    while len(train_indices) +len(test_indices) < len(df):
        if ind not in train_indices and ind not in test_indices:
            if ind % frac != 0:
                n_old = len(train_indices)
                train_indices.append(ind)
                train_indices = list(set(train_indices))

                while n_old != len(train_indices):
                    n_old = len(train_indices)

                    training_seqs= list(set(df["ESM1b"].loc[train_indices]))

                    train_indices = train_indices + (list(df.loc[df["ESM1b"].isin(training_seqs)].index))
                    train_indices = list(set(train_indices))
                
            else:
                n_old = len(test_indices)
                test_indices.append(ind)
                test_indices = list(set(test_indices))

                while n_old != len(test_indices):
                    n_old = len(test_indices)

                    testing_seqs= list(set(df["ESM1b"].loc[test_indices]))

                    test_indices = test_indices + (list(df.loc[df["ESM1b"].isin(testing_seqs)].index))
                    test_indices = list(set(test_indices))
                
        ind +=1
    return(df.loc[train_indices], df.loc[test_indices])

In [5]:
data_train2 = df_train.copy()
data_train2 = array_column_to_strings(data_train2, column = "ESM1b")

data_train2, df_fold = split_dataframe(df = data_train2, frac=5)
indices_fold1 = list(df_fold["index"])
print(len(data_train2), len(indices_fold1))#

data_train2, df_fold = split_dataframe(df = data_train2, frac=4)
indices_fold2 = list(df_fold["index"])
print(len(data_train2), len(indices_fold2))

data_train2, df_fold = split_dataframe(df = data_train2, frac=3)
indices_fold3 = list(df_fold["index"])
print(len(data_train2), len(indices_fold3))

data_train2, df_fold = split_dataframe(df = data_train2, frac=2)
indices_fold4 = list(df_fold["index"])
indices_fold5 = list(data_train2["index"])
print(len(data_train2), len(indices_fold4))


fold_indices = [indices_fold1, indices_fold2, indices_fold3, indices_fold4, indices_fold5]

train_indices = [[], [], [], [], []]
test_indices = [[], [], [], [], []]

for i in range(5):
    for j in range(5):
        if i != j:
            train_indices[i] = train_indices[i] + fold_indices[j]
            
    test_indices[i] = fold_indices[i]
    
np.save(join(CURRENT_DIR, ".." ,"data","splits", "CV_train_indices.npy"), train_indices)
np.save(join(CURRENT_DIR, ".." ,"data","splits", "CV_test_indices.npy"), test_indices)

43497 10678
32572 10925
21656 10916
10917 10739


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.

In [11]:
train_indices = list(np.load(join(CURRENT_DIR, ".." ,"data","splits", "CV_train_indices.npy"),  allow_pickle=True))
test_indices = list(np.load(join(CURRENT_DIR, ".." ,"data","splits", "CV_test_indices.npy"),  allow_pickle=True))
print(len(test_indices[0]) + len(train_indices[0]))
train_indices = [ [idx for idx in row if idx < train_length] for row in train_indices]
test_indices = [ [idx for idx in row if idx < train_length] for row in test_indices]
len(train_indices[0])
len(test_indices[0]) + len(train_indices[0])

54175


52619

### Creating numpy arrays with input vectors and output variables

In [6]:
def create_input_and_output_data(df):
    X = ();
    y = ();
    
    for ind in df.index:
        emb = df["ESM1b_ts"][ind]
        ecfp = np.array(df["unimol"][ind]).squeeze()
                
        X = X +(np.concatenate([ecfp, emb]), );
        y = y + (df["Binding"][ind], );

    return(X,y)

train_X, train_y =  create_input_and_output_data(df = df_train)
test_X, test_y =  create_input_and_output_data(df = df_test)


feature_names =  ["ECFP_" + str(i) for i in range(1024)]
feature_names = feature_names + ["ESM1b_ts_" + str(i) for i in range(1280)]

train_X = np.array(train_X)
test_X  = np.array(test_X)

train_y = np.array(train_y)
test_y  = np.array(test_y)

In [7]:
# 检查 train_X 中第一个数据点的长度
if 'train_X' in locals() and len(train_X) > 0:
    first_sample_length = len(train_X[0])
    print(f"Length of a single data point in train_X: {first_sample_length}")
    
    # (可选) 检查所有数据点的长度是否一致
    all_lengths_consistent = all(len(sample) == first_sample_length for sample in train_X)
    if all_lengths_consistent:
        print("All data points in train_X have a consistent length.")
    else:
        print("Warning: Data points in train_X have inconsistent lengths.")
else:
    print("train_X is not defined or is empty. Please run the data creation cell first.")


Length of a single data point in train_X: 2048
All data points in train_X have a consistent length.


In [8]:
classes = np.unique(train_y)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=train_y)
class_weights = dict(zip(classes, weights))

# normalize the features
scaler = StandardScaler()
scaler.fit(train_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)

## 2. Training and validation machine learning models

### (a) Logistic Regression

#### (ii) Performing hyperparameter optimization

In [9]:
def cross_validation_neg_acc_logistic_regression(param):
    param['solver'] = 'liblinear'
    param['class_weight'] = class_weights
    
    loss = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]        
    
        clf = LogisticRegression(max_iter=20, penalty = param["penalty"],
                                C = param["C"], solver = param['solver'],
                                class_weight= param["class_weight"]
                                ).fit(train_X[train_index], train_y[train_index])
        y_valid_pred = np.round(clf.predict(train_X[test_index]))
        validation_y = train_y[test_index]
    
        false_positive = 100*(1-np.mean(np.array(validation_y)[y_valid_pred == 1]))
        false_negative = 100*(np.mean(np.array(validation_y)[y_valid_pred == 0]))
        logging.info("False positive rate: " + str(false_positive)+ "; False negative rate: " + str(false_negative))
        loss.append(2*(false_negative**2) + false_positive**1.3)
    return(np.mean(loss))

#Defining search space for hyperparameter optimization
space_logistic_regression = {'C': hp.choice('C', [1, 10, 100, 1000]),
                            'penalty': hp.choice('penalty', ['l1', 'l2'])}

Performing a random grid search:

In [10]:
'''trials = Trials()

for i in range(1,10):
    best = fmin(fn = cross_validation_neg_acc_logistic_regression, space = space_logistic_regression,
                algo = rand.suggest, max_evals = i, trials = trials)
    print(i)
    print(trials.best_trial["result"]["loss"])
    print(trials.argmin)''';

Best set of hyperparameters:

In [11]:
'''param = {"C" : 1,
        "penalty" : "l2"}

param['solver'] = 'liblinear'
param['class_weight'] = class_weights
np.unique(train_y[test_indices[0]])'''

'param = {"C" : 1,\n        "penalty" : "l2"}\n\nparam[\'solver\'] = \'liblinear\'\nparam[\'class_weight\'] = class_weights\nnp.unique(train_y[test_indices[0]])'

#### (iii) Repeating 5-fold CV for best set of hyperparameters

In [12]:
'''loss = []
accuracy = []
ROC_AUC = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    clf = LogisticRegression(max_iter=250, penalty = param["penalty"],
                                C = param["C"], solver = param['solver'],
                                class_weight= param["class_weight"]
                                ).fit(train_X[train_index], train_y[train_index])
    y_valid_pred = np.round(clf.predict(train_X[test_index]))
    validation_y = train_y[test_index]

    #calculate loss:
    false_positive = 100*(1-np.mean(np.array(validation_y)[y_valid_pred == 1]))
    false_negative = 100*(np.mean(np.array(validation_y)[y_valid_pred == 0]))
    logging.info("False positive rate: " + str(false_positive)+ "; False negative rate: " + str(false_negative))
    loss.append(2*(false_negative**2) + false_positive**1.3)
    #calculate accuracy:
    accuracy.append(np.mean(y_valid_pred == np.array(validation_y)))
    #calculate ROC-AUC score:
    ROC_AUC.append(roc_auc_score(np.array(validation_y), clf.predict_proba(train_X[test_index])[:, 1]))
    
print("Loss values: %s" %loss) 
print("Accuracies: %s" %accuracy)
print("ROC-AUC scores: %s" %ROC_AUC)

np.save(join(CURRENT_DIR, ".." ,"data", "training_results", "acc_CV_LR.npy"), np.array(accuracy))
np.save(join(CURRENT_DIR, ".." ,"data", "training_results", "loss_CV_LR.npy"), np.array(loss))
np.save(join(CURRENT_DIR, ".." ,"data", "training_results", "ROC_AUC_CV_LR.npy"), np.array(ROC_AUC))'''

'loss = []\naccuracy = []\nROC_AUC = []\n\nfor i in range(5):\n    train_index, test_index  = train_indices[i], test_indices[i]\n    clf = LogisticRegression(max_iter=250, penalty = param["penalty"],\n                                C = param["C"], solver = param[\'solver\'],\n                                class_weight= param["class_weight"]\n                                ).fit(train_X[train_index], train_y[train_index])\n    y_valid_pred = np.round(clf.predict(train_X[test_index]))\n    validation_y = train_y[test_index]\n\n    #calculate loss:\n    false_positive = 100*(1-np.mean(np.array(validation_y)[y_valid_pred == 1]))\n    false_negative = 100*(np.mean(np.array(validation_y)[y_valid_pred == 0]))\n    logging.info("False positive rate: " + str(false_positive)+ "; False negative rate: " + str(false_negative))\n    loss.append(2*(false_negative**2) + false_positive**1.3)\n    #calculate accuracy:\n    accuracy.append(np.mean(y_valid_pred == np.array(validation_y)))\n    #calcul

#### (iv) 3. Training and validating the final model
Training the model and validating it on the test set:

In [13]:
'''clf = LogisticRegression(max_iter=250, penalty = param["penalty"],
                                C = param["C"], solver = param['solver'],
                                class_weight= param["class_weight"]
                                ).fit(train_X, train_y)
y_test_pred = np.round(clf.predict(test_X))
acc_test = np.mean(y_test_pred == np.array(test_y))
roc_auc = roc_auc_score(np.array(test_y), clf.predict_proba(test_X)[:, 1])
mcc = matthews_corrcoef(np.array(test_y), y_test_pred)

print("Accuracy on test set: %s, ROC-AUC score for test set: %s, MCC: %s"  % (acc_test, roc_auc, mcc))'''

'clf = LogisticRegression(max_iter=250, penalty = param["penalty"],\n                                C = param["C"], solver = param[\'solver\'],\n                                class_weight= param["class_weight"]\n                                ).fit(train_X, train_y)\ny_test_pred = np.round(clf.predict(test_X))\nacc_test = np.mean(y_test_pred == np.array(test_y))\nroc_auc = roc_auc_score(np.array(test_y), clf.predict_proba(test_X)[:, 1])\nmcc = matthews_corrcoef(np.array(test_y), y_test_pred)\n\nprint("Accuracy on test set: %s, ROC-AUC score for test set: %s, MCC: %s"  % (acc_test, roc_auc, mcc))'

### (b) Random forest

In [14]:
def cross_validation_neg_acc_random_forest(param):
    param['solver'] = 'liblinear'
    param['class_weight'] = class_weights
    
    loss = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]        
    
        clf = RandomForestClassifier(n_estimators = param["n_estimators"],
                                class_weight= param["class_weight"]
                                ).fit(train_X[train_index], train_y[train_index])
        y_valid_pred = np.round(clf.predict(train_X[test_index]))
        validation_y = train_y[test_index]
    
        false_positive = 100*(1-np.mean(np.array(validation_y)[y_valid_pred == 1]))
        false_negative = 100*(np.mean(np.array(validation_y)[y_valid_pred == 0]))
        logging.info("False positive rate: " + str(false_positive)+ "; False negative rate: " + str(false_negative))
        loss.append(2*(false_negative**2) + false_positive**1.3)
    return(np.mean(loss))

#Defining search space for hyperparameter optimization
space_random_forest = {'n_estimators': hp.choice('n_estimators', [100, 200, 300])}

Performing a random grid search:

In [15]:
'''trials = Trials()

for i in range(1,10):
    best = fmin(fn = cross_validation_neg_acc_random_forest, space = space_random_forest,
                algo = rand.suggest, max_evals = i, trials = trials)
    print(i)
    print(trials.best_trial["result"]["loss"])
    print(trials.argmin)''';

Best set of hyperparameters:

In [16]:
param = {"n_estimators" : 100}
param['class_weight'] = class_weights

In [17]:
loss = []
accuracy = []
ROC_AUC = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    clf = RandomForestClassifier(n_estimators = param["n_estimators"],
                                class_weight= param["class_weight"]
                                ).fit(train_X[train_index], train_y[train_index])
    y_valid_pred = np.round(clf.predict(train_X[test_index]))
    validation_y = train_y[test_index]

    #calculate loss:
    false_positive = 100*(1-np.mean(np.array(validation_y)[y_valid_pred == 1]))
    false_negative = 100*(np.mean(np.array(validation_y)[y_valid_pred == 0]))
    logging.info("False positive rate: " + str(false_positive)+ "; False negative rate: " + str(false_negative))
    loss.append(2*(false_negative**2) + false_positive**1.3)
    #calculate accuracy:
    accuracy.append(np.mean(y_valid_pred == np.array(validation_y)))
    #calculate ROC-AUC score:
    ROC_AUC.append(roc_auc_score(np.array(validation_y), clf.predict_proba(train_X[test_index])[:, 1]))
    
print("Loss values: %s" %loss) 
print("Accuracies: %s" %accuracy)
print("ROC-AUC scores: %s" %ROC_AUC)

np.save(join(CURRENT_DIR, ".." ,"data", "training_results", "acc_CV_random_forest.npy"), np.array(accuracy))
np.save(join(CURRENT_DIR, ".." ,"data", "training_results", "loss_CV_random_forest.npy"), np.array(loss))
np.save(join(CURRENT_DIR, ".." ,"data", "training_results", "ROC_AUC_CV_random_forest.npy"), np.array(ROC_AUC))

Loss values: [300.3879056626604, 270.2830773331824, 271.7239055676095, 263.06719415399624, 262.7506355402403]
Accuracies: [0.8867013372956909, 0.8934669636228656, 0.8927899686520376, 0.8950461796809404, 0.8922670191672174]
ROC-AUC scores: [0.9327843302967208, 0.9342638103832339, 0.9373403389791934, 0.9374827542536525, 0.9315660871475806]


#### (iv) 3. Training and validating the final model
Training the model and validating it on the test set:

In [18]:
clf = RandomForestClassifier(n_estimators = param["n_estimators"],
                                class_weight= param["class_weight"]
                                ).fit(train_X, train_y)
y_test_pred = np.round(clf.predict(test_X))
acc_test = np.mean(y_test_pred == np.array(test_y))
roc_auc = roc_auc_score(np.array(test_y), clf.predict_proba(test_X)[:, 1])
mcc = matthews_corrcoef(np.array(test_y), y_test_pred)

print("Accuracy on test set: %s, ROC-AUC score for test set: %s, MCC: %s"  % (acc_test, roc_auc, mcc))

Accuracy on test set: 0.9006459550907413, ROC-AUC score for test set: 0.9558865844969896, MCC: 0.7356092858899965
